# Import Necessary Dependencies

In [16]:
import pandas as pd
import numpy as np
import math

# Upload Local Data and Condense

In [17]:
# California Data Exchange Center(CDEC) Water Resource data
df_MRN_h = pd.read_csv("data/CDEC/MRN/ytd_hr_humidity.csv")
df_MRN_p = pd.read_csv("data/CDEC/MRN/ytd_hr_precipitation.csv")
df_MRN_t = pd.read_csv("data/CDEC/MRN/ytd_hr_temp.csv")

In [18]:
df_MRN_h = df_MRN_h.rename(columns={'VALUE': 'REL_HUM_VALUE'})
df_MRN_p = df_MRN_p.rename(columns={'VALUE': 'RAIN_VALUE'})
df_MRN_t = df_MRN_t.rename(columns={'VALUE': 'TEMP_VALUE'})

df_MRN_t['REL_HUM_VALUE'] = df_MRN_h.get("REL_HUM_VALUE")
df_MRN_t['RAIN_VALUE'] = df_MRN_p.get("RAIN_VALUE")
df_MRN_t

,STATION_ID,DURATION,SENSOR_NUMBER,SENSOR_TYPE,DATE TIME,OBS DATE,TEMP_VALUE,DATA_FLAG,UNITS,REL_HUM_VALUE,RAIN_VALUE
0,MRN,H,4,TEMP,20240101 0000,20240101 0000,36,,DEG F,94,2.85
1,MRN,H,4,TEMP,20240101 0100,20240101 0100,35,,DEG F,96,2.85
2,MRN,H,4,TEMP,20240101 0200,20240101 0200,38,,DEG F,89,2.85
3,MRN,H,4,TEMP,20240101 0300,20240101 0300,36,,DEG F,92,2.85
4,MRN,H,4,TEMP,20240101 0400,20240101 0400,35,,DEG F,92,2.85
...,...,...,...,...,...,...,...,...,...,...,...
2635,MRN,H,4,TEMP,20240419 2000,20240419 2000,55,,DEG F,62,15.48
2636,MRN,H,4,TEMP,20240419 2100,20240419 2100,52,,DEG F,68,15.48
2637,MRN,H,4,TEMP,20240419 2200,20240419 2200,50,,DEG F,72,15.48
2638,MRN,H,4,TEMP,20240419 2300,20240419 2300,48,,DEG F,79,15.48


In [19]:
df_MRN_t.dropna(inplace=True);

In [20]:
def t_c(temp):
    if temp == "---":
        return 0
    else:
        return temp
df_MRN_t['TEMP_VALUE'] = df_MRN_t.get("TEMP_VALUE").apply(t_c)

In [23]:
def haversine_distance(coord1, coord2):
    lat, lng = coord1
    lat0, lng0 = coord2
    deglen = 110.25
    x = lat - lat0
    y = (lng - lng0)*math.cos(lat0)
    return deglen*np.sqrt(x*x + y*y)

def kmeans(global_coord, k_coords):
    ans = []
    #change global data to toby
    for x in global_coord:
        dist = [math.inf,0]
        for j,k in enumerate(k_coords):
            dist_s = haversine_distance(k, x)
            if dist_s < dist[0]:
                dist[0] = dist_s
                dist[1] = j
        ans.append(dist[1])
    return ans

# Augmented Current Icing Product Model

In [21]:
def augmented_cip_model(station_data, cloud_top_temperature=-27, cloud_mask=0.8):
    """
    Applies a augmented CIP model to weather station data with cloud top temperature and cloud mask.

    Args:
        station_data (pandas.DataFrame): DataFrame containing temperature, humidity, and rain values.
        cloud_top_temperature (float): Cloud top temperature in degrees Fahrenheit (default: -27).
        cloud_mask (float): Cloud mask value (default: 0.8).

    Returns:
        pandas.DataFrame: DataFrame with an added "res" column containing icing potential percentages.
    """

    def sigmoid(x):
        return 1 / (1 + math.exp(-x))
    
    station_data['TEMP_VALUE_C'] = (pd.to_numeric(station_data['TEMP_VALUE']) - 32) * 5/9

    # Fuzzy Logic Functions
    def tmap(temperature):
        if temperature < 30:
            return 0.0
        elif 30 <= temperature <= 45:
            return (temperature + 25) / 10
        elif 45 < temperature <= 50:
            return 1.0
        elif 50 < temperature <= 60:
            return (-temperature) / 3
        else:
            return 0.0

    def cttmap(ctt):
        if ctt < -30:
            return 0.1
        elif -30 <= ctt <= -12:
            return (ctt + 30) / 18
        elif -12 < ctt <= 8:
            return 1.0
        elif 8 < ctt <= 12:
            return (12 - ctt) / 4
        else:
            return 0.5

    def rhmap(rh):
        if rh < 70:
            return 0.0
        elif 70 <= rh <= 90:
            return (rh - 70) / 20
        else:
            return 1.0
            return 1.0

    def calculate_icing_potential(row):
        if row['TEMP_VALUE_C'] == "---":
            temperature = 0
        else:
            temperature = int(row['TEMP_VALUE'])
        if row['RAIN_VALUE'] == "---":
            rain = 0
        else:
            rain = float(row['RAIN_VALUE'])
        if row['REL_HUM_VALUE'] == "---":
            humidity = 0
        else:
            humidity = int(row['REL_HUM_VALUE'])
        
        if cloud_mask >= 0.5 and rain > 0:  
            if temperature >= 0:  
                return sigmoid(tmap(temperature) * cttmap(cloud_top_temperature))
            else:  
                return sigmoid(0.8 * tmap(temperature) * cttmap(cloud_top_temperature))
        elif cloud_mask >= 0.5: 
            return sigmoid(tmap(temperature) * cttmap(cloud_top_temperature) * rhmap(humidity))
        else:
            return 0.0

    station_data['res'] = station_data.apply(calculate_icing_potential, axis=1)

    # convert icing potential to percentage
    station_data['res'] = station_data['res'] * 100

    return station_data

In [22]:
res = augmented_cip_model(df_MRN_t)